### Download and Preprocess data

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip

--2019-07-22 05:02:11--  https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306636009 (292M) [application/x-httpd-php]
Saving to: ‘OpportunityUCIDataset.zip’

OpportunityUCIDatas 100%[===================>] 292.43M  93.1MB/s    in 3.3s    

2019-07-22 05:02:15 (87.6 MB/s) - ‘OpportunityUCIDataset.zip’ saved [306636009/306636009]



In [5]:
!python3 /content/preprocess_data.py -i /content/OpportunityUCIDataset.zip -o oppChallenge_gestures.data

Checking dataset /content/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.da

### Setup

In [0]:
import numpy as np
import _pickle as cp
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras import optimizers
from keras.utils import np_utils

from sliding_window import sliding_window

NB_SENSOR_CHANNELS = 113
NUM_CLASSES = 18
SLIDING_WINDOW_LENGTH = 24
FINAL_SEQUENCE_LENGTH = 8
SLIDING_WINDOW_STEP = 12
BATCH_SIZE = 100
NUM_FILTERS = 64
FILTER_SIZE = 5
NUM_UNITS_LSTM = 128

### Load the sensor data

In [8]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('/content/oppChallenge_gestures.data')

Loading data...
 ..from file /content/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)


### Segmentation and Reshaping

In [9]:
assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x, (ws, data_x.shape[1]), (ss, 1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y, ws, ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

# Data is reshaped
X_train = X_train.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
y_train = np_utils.to_categorical(y_train) # one-hot encoding
y_test = np_utils.to_categorical(y_test) # one-hot encoding

print(" ..after sliding and reshaping, train data: inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
print(" ..after sliding and reshaping, test data : inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

 ..after sliding and reshaping, train data: inputs (46495, 24, 113), targets (46495, 18)
 ..after sliding and reshaping, test data : inputs (9894, 24, 113), targets (9894, 18)


![](https://github.com/yminoh/DeepConvLSTM_Python3/raw/master/architecture_of_DeepConvLSTM.png)  
<div style="text-align: center">Figure 3 from Ordonez and Roggen, 2016</div>

### Build and train the Kears layers

In [10]:
rmp = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model = Sequential()
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, activation='relu', kernel_initializer='orthogonal',
                 input_shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS), 
                 name='Conv1D_1'))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_2'))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_3'))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, activation='relu', kernel_initializer='orthogonal',
                 name='Conv1D_4'))
model.add(LSTM(NUM_UNITS_LSTM, return_sequences=True, kernel_initializer='orthogonal', 
               name='LSTM_1'))
model.add(LSTM(NUM_UNITS_LSTM, return_sequences=True, kernel_initializer='orthogonal', 
               name='LSTM_2'))
model.add(Flatten(name='Flatten'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(NUM_CLASSES, activation='softmax', kernel_initializer='orthogonal', 
                name='Output'))

model.compile(loss='categorical_crossentropy', optimizer=rmp, metrics=['accuracy'])

print(model.summary())

W0722 05:10:01.045839 139670424545152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0722 05:10:01.048480 139670424545152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 05:10:01.752909 139670424545152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0722 05:10:01.765064 139670424545152 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a futur

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1D_1 (Conv1D)            (None, 20, 64)            36224     
_________________________________________________________________
Conv1D_2 (Conv1D)            (None, 16, 64)            20544     
_________________________________________________________________
Conv1D_3 (Conv1D)            (None, 12, 64)            20544     
_________________________________________________________________
Conv1D_4 (Conv1D)            (None, 8, 64)             20544     
_________________________________________________________________
LSTM_1 (LSTM)                (None, 8, 128)            98816     
_________________________________________________________________
LSTM_2 (LSTM)                (None, 8, 128)            131584    
_________________________________________________________________
Flatten (Flatten)            (None, 1024)              0         
__________

![](https://github.com/yminoh/DeepConvLSTM_Python3/raw/master/Table1.png)  
<div style="text-align: center">Table 1 from Ordonez and Roggen, 2016</div>

In [11]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=5, batch_size=BATCH_SIZE, verbose=1)

W0722 05:10:13.949736 139670424545152 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0722 05:10:15.753759 139670424545152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 46495 samples, validate on 9894 samples
Epoch 1/5
46495/46495 [==============================] - 47s 1ms/step - loss: 1.3479 - acc: 0.6927 - val_loss: 0.8586 - val_acc: 0.8325
Epoch 2/5
46495/46495 [==============================] - 44s 947us/step - loss: 1.1919 - acc: 0.6943 - val_loss: 0.9553 - val_acc: 0.8312
Epoch 3/5
46495/46495 [==============================] - 45s 961us/step - loss: 1.1228 - acc: 0.6952 - val_loss: 0.7463 - val_acc: 0.8323
Epoch 4/5
46495/46495 [==============================] - 44s 957us/step - loss: 1.0438 - acc: 0.6970 - val_loss: 0.8926 - val_acc: 0.7835
Epoch 5/5
46495/46495 [==============================] - 45s 963us/step - loss: 0.9738 - acc: 0.7008 - val_loss: 0.6546 - val_acc: 0.8383


### Evaluate the model

In [12]:
test_pred = np.argmax(model.predict(X_test), axis=1)
test_true = np.argmax(y_test, axis=1)
np.unique(test_pred)

array([ 0,  2,  3, 13, 14, 16])

In [13]:
import sklearn.metrics as metrics
print("\tTest fscore:\t{:.4f} ".format(metrics.f1_score(test_true, test_pred, average='weighted')))

	Test fscore:	0.7812 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
